## Soley Narrator agent response

##### Create Narrator

In [ ]:
SEED=42

model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key, seed=SEED, temperature=TEMPERATURE)

narrator = AssistantAgent(
    name="Narrator",
    system_message="You're a writer. Please generate your asnwer based on task and critic's feedback ",
    model_client=model_client,
    reflect_on_tool_use=False,
)

##### Soley Narrator response

In [ ]:
async def assistant_run() -> None:
    response = await narrator.on_messages(
        [TextMessage(content=prompt, source="user")],
        cancellation_token=CancellationToken(),
    )
    text=response.chat_message.content
    #splitted_sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [text] 

result_list = await assistant_run()
#for sentence in result_list:
    #print(sentence)


## Using T's code to extract differences & I defined a function to summarize the errors.

Extract values from narrative

In [ ]:
extractor=ExtractionModel(ds_info=ds_info, llm=gpt)
extraction=extractor.generate_extractions(Timmour_story)
extraction
#Because generate_extractions(...) returns a list of dicts—one dict per narrative—the 0th element is simply the dictionary for the first narrative. If you have multiple narratives, you’ll get multiple dictionaries in extraction, one for each entry. Accessing [0] is how you retrieve the very first dictionary in that list.


Extracted story 1/1 with gpt-4o


[{'Mjob_other': {'rank': 0,
   'sign': -1,
   'value': None,
   'assumption': "The model associates the mother's job being categorized as 'other' with less stable or lower-income jobs, potentially impacting the student's academic support at home."},
  'failures': {'rank': 1,
   'sign': 1,
   'value': 0,
   'assumption': 'A clean academic record, indicated by no past class failures, often correlates with future success.'},
  'goout': {'rank': 2,
   'sign': 1,
   'value': None,
   'assumption': 'A below-average tendency to go out with friends might indicate a more focused approach to studies.'},
  'sex': {'rank': 3,
   'sign': 1,
   'value': 1,
   'assumption': 'There is a pattern in the dataset where male students have a slightly higher pass rate.'}}]

In [ ]:
prompt2=extractor.generate_prompt(Timmour_story)
print(prompt2)

generate differences

In [ ]:
rank_diff, sign_diff , value_diff, real_rank, extracted_rank=extractor.get_diff(extraction[0],generator.explanation_list[0])
#compare the difference between generated SHAP table from GenerationModel (explanation_list) vs extracted features from ExtractionModel (extraction[0])

In [ ]:
df_diff = pd.DataFrame({
    "rank": rank_diff,
    "sign": sign_diff,
    "value": value_diff
})

generate feedback

In [ ]:
def check_ones_and_none(df: pd.DataFrame,features_dict: dict) -> str:
    """
    Checks each row of df for:
      - Values not equal to 0 (treated as an error)
      - Values that are None/NaN (didn't extract any value)

    Integrates feature names from features_dict based on their 'rank'.
    If no issues found, returns a '100.0% correct' message.
    """
    # Build a mapping from row index (rank) to feature name
    # Example: rank_to_feature[0] = 'failures'
    rank_to_feature = {}
    for feat_name, feat_info in features_dict.items():
        rank = feat_info.get('rank')
        if rank is not None:
            rank_to_feature[rank] = feat_name

    messages = []
    found_issues = False

    for idx, row in df.iterrows():
        # Get the feature name for this row's rank
        # If a rank is missing in the dictionary, default to "Unknown"
        feature_name = rank_to_feature.get(idx, f"Unknown_{idx}")
        
        # Track columns with 1 and with None (NaN)
        cols_with_one = []
        cols_with_none = []
        
        for col, val in row.items():
            if val != 0:
                cols_with_one.append(col)
            elif pd.isnull(val):
                # This captures both None and NaN
                cols_with_none.append(col)

        # If we found any 1 in this row, report it
        if cols_with_one:
            found_issues = True
            messages.append(f"For row {feature_name}, column {cols_with_one} contains an error.")
        
        # If we found any None/NaN in this row, report it
        if cols_with_none:
            found_issues = True
            messages.append(f"For row {feature_name}, column {cols_with_none} didn't extract any value.")

    # If we never found a 1 or a None/NaN, everything is 0
    if not found_issues:
        return "After checking, the narrative is 100% correct."
    
    return "\n".join(messages)

feedback = check_ones_and_none(df_diff, extraction[0])
print(feedback)

For row Mjob_other, column ['value'] didn't extract any value.
For row goout, column ['value'] didn't extract any value.


In [ ]:
async def main() -> None:

    SEED=42
    model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key, seed=SEED, temperature=TEMPERATURE)

    narrator = AssistantAgent(
        name="Narrator",
        system_message="You're a writer. Please generate your asnwer based on task and critic's feedback",
        model_client=model_client,
        reflect_on_tool_use=False,
    )

    critic = AssistantAgent(
        name="Critic",
        system_message="You are a critic. You review the content from"
                    "the Narrator and provide constructive "
                    "feedback to help improve the quality of the content.",
        model_client=model_client,
        reflect_on_tool_use=False,
    )
    # The termination condition is a combination of text termination and max message termination, either of which will cause the chat to terminate.
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(4)

    summary_method="last_msg"
    # The group chat will alternate between the assistant and the code executor.
    group_chat = RoundRobinGroupChat([narrator, critic], termination_condition=termination)

    # `run_stream` returns an async generator to stream the intermediate messages.
    stream = group_chat.run_stream(task=prompt)
    
    # `Console` is a simple UI to display the stream.
    await Console(stream)

await main()

## Other codes

define the order of groupchats

In [ ]:
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],#Only specific agents are allowed to talk to certain others,
        # only user_proxy and planner are alowed to response writer.
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed", 

nested group chat

In [ ]:
import asyncio
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent, SocietyOfMindAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination


async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key, seed=SEED, temperature=TEMPERATURE)

    agent1 = AssistantAgent("assistant1", model_client=model_client, system_message="You are a writer, write well.")
    agent2 = AssistantAgent(
        "assistant2",
        model_client=model_client,
        system_message="You are an editor, provide critical feedback. Respond with 'APPROVE' if the text addresses all feedbacks.",
    )
    inner_termination = TextMentionTermination("APPROVE")
    inner_team = RoundRobinGroupChat([agent1, agent2], termination_condition=inner_termination)

    society_of_mind_agent = SocietyOfMindAgent("society_of_mind", team=inner_team, model_client=model_client)

    agent3 = AssistantAgent(
        "assistant3", model_client=model_client, system_message="Translate the text to Spanish."
    )
    team = RoundRobinGroupChat([society_of_mind_agent, agent3], max_turns=2)

    stream = team.run_stream(task="Write a short story with a surprising ending.")
    await Console(stream)


await main()


In [ ]:
import re
Timmour_story=generator.generate_stories(trained_model,x,y)
narrative_split=re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', Timmour_story[0])
for sentence in narrative_split:
    print(sentence)

In [ ]:
prompt2= extractor.generate_prompt (Timmour_story)
print(prompt2)
# prompt2_split=re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', prompt2)
# for sentence in prompt2_split:
#     print(sentence)

In [ ]:
from shapnarrative_metrics.metrics import faithfulness
test= faithfulness.average_zero(rank_diff)

In [7]:
import yaml
with open("../config/keys.yaml") as f:
    dict=yaml.safe_load(f)
api_key = dict["API_keys"]["OpenAI"]

In [3]:
import sys
import os

sys.path.append("../SHAPnarrative-metrics")

Use check ones and none to test if after manipulate, the function can completely identify where is wrong

In [ ]:
TEMPERATURE=0
MANIP=True

gpt = llm_wrappers.GptApi(api_key, model="gpt-4o", system_role="You are a teacher that explains AI predictions.", temperature=TEMPERATURE)
generator=GenerationModel(ds_info=ds_info, llm=gpt)
generator.gen_variables(trained_model,x,y,tree=True)
shap_df=pd.DataFrame(generator.explanation_list[0].head(4))
prompt = generator.generate_story_prompt(0,prompt_type="long",manipulate=MANIP)
print(prompt)

Timmour_story=generator.generate_stories(trained_model,x,y,prompt_type="long",manipulate=MANIP)
extractor=ExtractionModel(ds_info=ds_info, llm=gpt)
extraction=extractor.generate_extractions(Timmour_story)
extraction

def check_ones_and_none(df: pd.DataFrame,features_dict: dict) -> str:
    """
    Checks each row of df for:
      - Values not equal to 0 (treated as an error)
      - Values that are None/NaN (didn't extract any value)

    Integrates feature names from features_dict based on their 'rank'.
    If no issues found, returns a '100.0% correct' message.
    """
    # Build a mapping from row index (rank) to feature name
    # Example: rank_to_feature[0] = 'failures'
    rank_to_feature = {}
    for feat_name, feat_info in features_dict.items():
        rank = feat_info.get('rank')
        if rank is not None:
            rank_to_feature[rank] = feat_name

    messages = []
    found_issues = False

    for idx, row in df.iterrows():
        # Get the feature name for this row's rank
        # If a rank is missing in the dictionary, default to "Unknown"
        feature_name = rank_to_feature.get(idx, f"Unknown_{idx}")
        
        # Track columns with 1 and with None (NaN)
        cols_with_one = []
        cols_with_none = []
        
        for col, val in row.items():
            if val != 0:
                cols_with_one.append(col)
            elif pd.isnull(val):
                # This captures both None and NaN
                cols_with_none.append(col)

        # If we found any 1 in this row, report it
        if cols_with_one:
            found_issues = True
            messages.append(f"For row {feature_name}, column {cols_with_one} contains an error.")
        
        # If we found any None/NaN in this row, report it
        if cols_with_none:
            found_issues = True
            messages.append(f"For row {feature_name}, column {cols_with_none} didn't extract any value.")

    # If we never found a 1 or a None/NaN, everything is 0
    if not found_issues:
        return "After checking, the narrative is 100% correct."
    
    return "\n".join(messages)

feedback = check_ones_and_none(df_diff, extraction[0])
print(feedback)

## Critic respond only with extracted features

In [ ]:
prompt = prompt

async def main() -> None:
    SEED = 42
    TEMPERATURE = 0.7  # Adjust as needed

    # Assuming you have these imports/definitions
    # from your_extraction_module import ExtractionModel, DatasetInfo
    
    # Initialize OpenAI client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o", 
        api_key=api_key, 
        seed=SEED, 
        temperature=TEMPERATURE
    )

    # Custom extension of AssistantAgent to include extraction functionality
    class ExtractorCriticAgent(AssistantAgent):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.extractor = None
            self.last_narrative = None

        def set_extractor(self, extractor_class, ds_info, llm):
            """
            Set up the extraction model
            
            Args:
                extractor_class: The ExtractionModel class
                ds_info: Dataset information
                llm: Language model client
            """
            self.extractor = extractor_class(ds_info=ds_info, llm=llm)

        def prepare_critic_input(self, narrative):
            """
            Generate extractions from the narrative
            
            Args:
                narrative (str): The narrative text to extract from
            
            Returns:
                dict: Extracted information
            """
            if not self.extractor:
                raise ValueError("Extractor not initialized. Call set_extractor first.")
            
            # Store the narrative for potential future use
            self.last_narrative = narrative
            
            # Generate extractions
            try:
                extraction = self.extractor.generate_extractions(narrative)
                return extraction
            except Exception as e:
                print(f"Extraction error: {e}")
                return {"error": str(e)}

    # Create the narrator agent
    narrator = AssistantAgent(
        name="Narrator", 
        system_message=system_message_narrator, 
        model_client=model_client,
        reflect_on_tool_use=False,
    )

    # Create the critic agent with extraction capabilities
    critic = ExtractorCriticAgent(
        name="Critic", 
        system_message=system_message_critic, 
        model_client=model_client,
        reflect_on_tool_use=False,
    )

    # Set up the extractor for the critic
    critic.set_extractor(ExtractionModel, ds_info, gpt)

    # Modify the group chat to pass narrative to extraction before critic review
    class CustomGroupChat(RoundRobinGroupChat):
        async def _process_message(self, sender, recipient):
            # If moving from narrator to critic, extract information first
            if sender.name == "Narrator" and recipient.name == "Critic":
                # Get the last message from the narrator
                last_message = sender.last_message()
                
                if last_message and 'content' in last_message:
                    # Prepare extraction for the critic
                    extraction = recipient.prepare_critic_input(last_message['content'])
                    
                    # Modify the message to include extraction
                    last_message['content'] += f"\n\nExtracted Information:\n{extraction}"
            
            # Continue with standard group chat processing
            return await super()._process_message(sender, recipient)

    # Create the group chat with custom processing
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(8)
    group_chat = CustomGroupChat(
        [narrator, critic], 
        termination_condition=termination
    )

    # Run the chat
    stream = group_chat.run_stream(task=prompt)
    await Console(stream)

# Run the main async function
if __name__ == "__main__":
  await main()

Validate if no data leakage and each round the comparison is between orginal shap_df and extracted information

In [ ]:
story='''The model predicted with 64% certainty that the student would pass the final year. The most significant factor contributing to this prediction was the father's education level, which, despite being below the average, had a positive impact. This suggests that even a primary education level can provide some foundational support for the student's academic journey. The student receiving extra educational support also played a crucial role in boosting the likelihood of passing, as it provided additional resources and guidance. The student's history of one past class failure, which is above the average, negatively impacted the prediction, indicating potential academic challenges. Lastly, the student's age of 19, while above the average, contributed positively, possibly reflecting a level of maturity and focus beneficial for academic success. Overall, the positive influences of the father's education, extra support, and maturity outweighed the negative impact of past failures, leading to the model's prediction that the student would pass.
'''

import re
narrative_split=re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', story)
for sentence in narrative_split:
    print(sentence)

extractor=ExtractionModel(ds_info=ds_info, llm=gpt)
extraction=extractor.generate_extractions([story])
extraction
# extraction_string=extractor.generate_prompt(story)
# print(extraction_string)

rank_diff, sign_diff , value_diff, real_rank, extracted_rank=extractor.get_diff(extraction[0],shap_df)
#compare the difference between generated SHAP table from GenerationModel (explanation_list) vs extracted features from ExtractionModel (extraction[0])'''

df_diff = pd.DataFrame({
    "rank": rank_diff,
    "sign": sign_diff,
    "value": value_diff
})
print(df_diff)

critic = ExtractorCriticAgent(
        name="Critic",
        system_message="""
You are a validation expert for SHAP narrative extraction analysis. 
Your feedback should focus ONLY on the generated validation_feedback.
Do not provide general comments or suggestions not directly based on the validation_feedback.
""",
        description="An agent that extracts and validates narrative faithfulness",
    )
    
    # Set up the models for the critic - THIS IS IMPORTANT
critic.set_models(
        extractor_class=ExtractionModel,
        generator_class=GenerationModel,
        ds_info=ds_info,
        llm=gpt
    )

errors= critic.prepare_critic_input([story],shap_df)
print (errors)

## Two agent system with defined well-structured format

In [ ]:
shap_df = shap_df

async def main() -> None:
    SEED = 42
    TEMPERATURE = 0

    # Initialize OpenAI client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o",
        api_key=api_key,
        seed=SEED,
        temperature=TEMPERATURE
    )
    
    # Create the narrator agent
    narrator = NarratorAgent(
        name="narrator",
        system_message=system_message_narrator,
        model_client=model_client,
        reflect_on_tool_use=False,
    )
    
    # Create the critic agent with advanced extraction capabilities
    extractorcritic = ExtractorCriticAgent(
        name="extractorcritic",
    )
    
    # Set up the models for the critic - THIS IS IMPORTANT
    extractorcritic.set_models(
        extractor_class=ExtractionModel,
        generator_class=GenerationModel,
        ds_info=ds_info,
        llm=gpt
    )

    extractorcritic.default_shap_df = shap_df 

    # Create the group chat with custom processing
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(12)
    
    group_chat = RoundRobinGroupChat(
        [narrator, extractorcritic],
        termination_condition=termination
    )
    
    # Run the chat
    stream = group_chat.run_stream(task=prompt)
    
    await Console(stream)

if __name__ == "__main__":
    await main()


## Three agent system with extracor critic and faithful critic

In [ ]:
shap_df = shap_df

async def main() -> None:
    SEED = 42
    TEMPERATURE = 0

    # Initialize OpenAI client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o",
        api_key=api_key,
        seed=SEED,
        temperature=TEMPERATURE
    )
    
    # Create the narrator agent
    narrator = NarratorAgent(
        name="narrator",
        system_message=narrator_sys_msg, 
        model_client=model_client,
        reflect_on_tool_use=False,
    )
    
    # Create the critic agent with advanced extraction capabilities
    faithful_evaluator = FaithfulEvaluatorAgent(
        name="faithful_evaluator",
    )
    
    # Set up the models for the critic - THIS IS IMPORTANT
    faithful_evaluator.set_models(
        extractor_class=ExtractionModel,
        generator_class=GenerationModel,
        ds_info=ds_info,
        llm=gpt
    )

    faithful_evaluator.default_shap_df = shap_df 

    faithful_critic = AssistantAgent(
        name="faithful_critic",
        system_message=faithful_critic_sys_msg,
        model_client=model_client,
        reflect_on_tool_use=False,
    )
    # Create the group chat with custom processing
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(20)
    
    group_chat = RoundRobinGroupChat(
        [narrator, faithful_evaluator, faithful_critic],
        model_client=model_client,
        termination_condition=termination,
    )

    # Run the chat
    await Console(group_chat.run_stream(task=prompt))

if __name__ == "__main__":
    await main()

## Four agent system with the coherence agent without any function call or perplexity.
After reaching 100% faithulness, the new perplexity start to work.

In [ ]:
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.messages import AgentEvent, ChatMessage
from selector_func import selector_func

import re

faithfulness_reached = False
coherence_phase_started = False

shap_df = shap_df

async def main() -> None:
    SEED = 42
    TEMPERATURE = 0

    # Initialize OpenAI client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o",
        api_key=api_key,
        seed=SEED,
        temperature=TEMPERATURE
    )
    
    # Create the narrator agent
    narrator = NarratorAgent(
        name="narrator",
        system_message=narrator_sys_msg, 
        model_client=model_client,
        reflect_on_tool_use=False,
    )
    
    # Create the critic agent with advanced extraction capabilities
    faithful_evaluator = FaithfulEvaluatorAgent(
        name="faithful_evaluator",
    )
    
    # Set up the models for the critic - THIS IS IMPORTANT
    faithful_evaluator.set_models(
        extractor_class=ExtractionModel,
        generator_class=GenerationModel,
        ds_info=ds_info,
        llm=gpt
    )

    faithful_evaluator.default_shap_df = shap_df 

    faithful_critic = AssistantAgent(
        name="faithful_critic",
        system_message=faithful_critic_sys_msg,
        model_client=model_client,
        reflect_on_tool_use=False,
    )

    coherenceagent = AssistantAgent(
            name="coherenceagent",
            system_message=coherence_sys_msg, 
            model_client=model_client,
            reflect_on_tool_use=False,
        )

    # Create the group chat with custom processing
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(20)
    
    group_chat = SelectorGroupChat(
        [narrator, faithful_evaluator, faithful_critic, coherenceagent],
        model_client=model_client,
        selector_func=selector_func,
        termination_condition=termination,
    )

    # Run the chat
    await Console(group_chat.run_stream(task=prompt))

if __name__ == "__main__":
    await main()